### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

### ✅ 1단계: 문서 분할 설정

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("[텍스트 분할기 생성 시작]")
# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,      # 법률 사례는 1200~1800자 권장
    chunk_overlap=300,    # 맥락 보존을 위해 200~400자 설정
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
print("[텍스트 분할기 생성 완료]")


[텍스트 분할기 생성 시작]
[텍스트 분할기 생성 완료]


### ✅ 2단계: 임베딩 모델 설정

In [3]:
from langchain_openai import OpenAIEmbeddings

try:
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large",
        dimensions=1536
    )
    print("✅ OpenAI 임베딩 모델 생성 성공")
except Exception as e:
    print(f"❌ 임베딩 모델 생성 실패: {e}")


✅ OpenAI 임베딩 모델 생성 성공


### ✅ 3단계: PDF 불러오기 및 문서 임베딩 → 벡터스토어 생성

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS

# PDF 파일 경로
pdf_path = "C:/mylangchain/langchain_basic/data/콘텐츠분쟁해결_사례.pdf"

# 1. 로딩
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# 2. 분할
documents = text_splitter.split_documents(pages)

# 3. 벡터 임베딩 & 저장소 생성
vectorstore = FAISS.from_documents(documents, embedding=embeddings)

# 4. 검색기 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

print(f"총 분할 문서 수: {len(documents)}개")


총 분할 문서 수: 104개


### ✅ 4단계: LLM 설정 (gpt-4o-mini 사용)

In [1]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

try:
    llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=2000
    )
    print("✅ ChatOpenAI LLM(gpt-4o) 객체 생성 성공")
except Exception as e:
    print(f"❌ LLM 객체 생성 실패: {e}")


✅ ChatOpenAI LLM(gpt-4o) 객체 생성 성공


### ✅ 5단계: 법률 자문 프롬프트 작성

In [ ]:
# 필요한 라이브러리 임포트
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# 1. 텍스트 분할기 설정 및 테스트
print("[텍스트 분할기 설정]")
print("=" * 50)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

print(f"✅ 청크 크기(chunk_size): {text_splitter._chunk_size}자")
print(f"✅ 오버랩(chunk_overlap): {text_splitter._chunk_overlap}자")
print(f"✅ 분할 구분자(separators): {text_splitter._separators[:4]}... (총 {len(text_splitter._separators)}개)")
print("=" * 50)

# 🔧 sample_text: 보기 좋게 들여쓰기 조정
sample_text = """
【사건개요】 
게임사 A는 2023년 5월 출시한 모바일 게임 내 프리미엄 아이템 확률 표시를 누락하여 이용자 1,200명이 집단 분쟁 제기.

【쟁점사항】
1. 확률 표시 의무 위반 여부
2. 피해 금액 산정 기준
3. 환불 범위 설정 문제

【처리결과】
게임사는 전체 이용자에게 아이템 비용 환불 및 10% 추가 보상 제공.

【사건개요】 
또 다른 게임사 B는 2024년 1월 출시한 게임에서 미성년자 결제 관련 분쟁이 발생함.

【쟁점사항】
1. 부모 동의 없는 결제 문제
2. 환불 정책의 적절성
3. 게임사 책임 범위

【처리경위】
게임사는 피해자들에게 개별 연락하여 환불 진행 중이며, 법적 조치도 검토함.

【처리결과】
일부 환불 완료, 추가 보상 여부는 추후 결정 예정.
"""

print("\n[테스트 분할 실행]")
print("=" * 50)
try:
    chunks = text_splitter.split_text(sample_text)
    print(f"✅ 분할 완료 (생성된 청크 수: {len(chunks)}개)")
except Exception as e:
    print(f"❌ 분할 실패: {str(e)}")

# 2. 프롬프트 템플릿 생성
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 실무적 해결방안을 단계별로 제시하세요
4. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)
print("\n✅ 프롬프트 템플릿 생성 완료")

# 3. ChatOpenAI LLM 객체 생성
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=2000
)
print("✅ ChatOpenAI LLM 객체 생성 완료")

# 4. 사용자 질문 입력
user_question = "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?"

# 5. 🔧 청크들 중 최대 길이 내에서 context 텍스트 구성 (필요시 다수 청크 결합 가능)
context_text = "\n\n".join(chunks)

# 6. 프롬프트 완성
final_prompt = prompt.format(context=context_text, question=user_question)

print("\n[완성된 프롬프트 내용]")
print("=" * 50)
print(final_prompt[:1500])  # 프롬프트 일부 출력

# 7. LLM 호출 및 응답 출력
print("\n[LLM 호출 중...]")
response = llm([HumanMessage(content=final_prompt)])  # 🔧 수정: 중복 import 제거

print("\n[LLM 응답]")
print("=" * 50)
print(response.content)


[텍스트 분할기 설정]
✅ 청크 크기(chunk_size): 3500자
✅ 오버랩(chunk_overlap): 300자
✅ 분할 구분자(separators): ['\n【사건개요】', '\n【쟁점사항】', '\n【처리경위】', '\n【처리결과】']... (총 10개)

[테스트 분할 실행]
✅ 분할 완료 (생성된 청크 수: 1개)

✅ 프롬프트 템플릿 생성 완료
✅ ChatOpenAI LLM 객체 생성 완료

[완성된 프롬프트 내용]

당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
【사건개요】 
게임사 A는 2023년 5월 출시한 모바일 게임 내 프리미엄 아이템 확률 표시를 누락하여 이용자 1,200명이 집단 분쟁 제기.

【쟁점사항】
1. 확률 표시 의무 위반 여부
2. 피해 금액 산정 기준
3. 환불 범위 설정 문제

【처리결과】
게임사는 전체 이용자에게 아이템 비용 환불 및 10% 추가 보상 제공.

【사건개요】 
또 다른 게임사 B는 2024년 1월 출시한 게임에서 미성년자 결제 관련 분쟁이 발생함.

【쟁점사항】
1. 부모 동의 없는 결제 문제
2. 환불 정책의 적절성
3. 게임사 책임 범위

【처리경위】
게임사는 피해자들에게 개별 연락하여 환불 진행 중이며, 법적 조치도 검토함.

【처리결과】
일부 환불 완료, 추가 보상 여부는 추후 결정 예정.

상담 내용: 미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 실무적 해결방안을 단계별로 제시하세요
4. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:


[LLM 호출 중...]

[LLM 응답]
미성년자가 부모 동의 없이 게임 아이템을 구매한 경우, 환불을 받

### ✅ 6단계: QA 체인 생성

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

print("QA 체인 생성 완료 ✅")


QA 체인 생성 완료 ✅


### ✅ 7단계: 테스트 질문 실행

In [20]:
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for i, q in enumerate(test_questions, 1):
    print(f"\n\n========================= 질문 {i} =========================")
    print(f"❓ 질문: {q}")

    # QA 체인 실행
    result = qa_chain.invoke({"query": q})

    # 답변 출력 (한 줄씩)
    print("\n✅ 생성된 답변:")
    for line in result["result"].split('\n'):
        print(line.strip())

    # 참조된 문서 정보 출력 (한 줄씩)
    print("\n📚 참고 문서 목록:")
    for j, doc in enumerate(result["source_documents"], 1):
        source = doc.metadata.get("source", "출처 정보 없음")
        preview = doc.page_content[:200].replace('\n', ' ').strip()
        print(f"  {j}. 문서 출처: {source}")
        print(f"     - 내용 미리보기: {preview} ...")




========================= 질문 1 =========================
❓ 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

✅ 생성된 답변:
온라인 게임에서 시스템 오류로 인해 아이템이 사라졌고, 게임회사가 복구를 거부하는 상황에 대해 다음과 같은 법률 조언을 드립니다.

### 1. 제시된 사례들을 근거로 한 분석
제시된 사례들을 살펴보면, 시스템 오류로 인해 아이템이 소멸된 경우와 관련하여 게임회사가 복구를 거부한 사례가 여러 건 있습니다. 예를 들어, **사례 1-가-1) 2006**에서는 시스템 오류로 인해 아이템이 소멸되었으나, 계정 명의자가 아닌 신청인이 복구를 요구하였고, 게임회사는 약관에 따라 복구를 거부했습니다. 또한, **사례 1-가-1) 2009**에서는 시스템 오류가 발생하지 않았다고 판단되어 복구가 거부된 사례도 있습니다.

### 2. 관련 법령 및 조항
- **민법 제250조**: 도품, 유실물에 대한 특례에 관한 규정으로, 도품이나 유실물의 경우 피해자는 반환을 청구할 수 있으나, 금전인 경우에는 그러하지 않다고 명시되어 있습니다.
- **게임회사의 약관**: 각 게임회사는 이용약관을 통해 계정 관리 및 아이템 복구에 대한 규정을 명시하고 있습니다. 이 약관은 법적 효력을 가지므로, 이를 준수해야 합니다.

### 3. 실무적 해결방안
1. **약관 검토**: 우선, 게임회사의 이용약관을 면밀히 검토하여 시스템 오류와 관련된 조항을 확인합니다. 특히, 아이템 복구에 대한 규정이 어떻게 되어 있는지 확인하는 것이 중요합니다.

2. **증거 수집**: 시스템 오류가 발생했음을 입증할 수 있는 증거(스크린샷, 오류 메시지 등)를 수집합니다. 이는 향후 분쟁 해결에 중요한 역할을 할 수 있습니다.

3. **정식 요청서 제출**: 게임회사에 정식으로 아이템 복구 요청서를 제출합니다. 이 요청서에는 시스템 오류로 인해 아이템이 소

### ✅ 8단계: (선택) 분쟁 유형 분류 함수

In [21]:
def classify_dispute_type(query):
    game_keywords = [
        "게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임",
        "접속", "패치", "서버", "버그", "결제", "인벤토리", "퀘스트",
        "보상", "랭킹", "게임머니", "게임사", "멀티플레이", "클래스",
        "업데이트", "이벤트", "친구", "길드전", "게임내재화"
    ]

    elearning_keywords = [
        "강의", "온라인교육", "이러닝", "수강", "환불", "화상교육",
        "수업", "강사", "학습", "콘텐츠", "교육비", "교재", "출석",
        "시험", "자격증", "학점", "플랫폼", "멤버십", "재수강",
        "과제", "퀴즈", "학습관리", "튜터링", "수강신청"
    ]

    web_keywords = [
        "웹사이트", "무료체험", "자동결제", "구독", "사이트",
        "회원가입", "탈퇴", "요금", "결제", "환불", "광고",
        "서비스", "이용약관", "개인정보", "쿠키", "프로모션",
        "업데이트", "고객센터", "접속장애", "비밀번호", "보안",
        "트래픽", "검색", "알림", "메일", "서버관리"
    ]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# 테스트 질문 리스트
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?",
    "길드에서 비매너 행위로 강퇴 당했습니다. 다시 복귀 할 수 있을까요?"
]

# 분쟁 유형 출력
for i, question in enumerate(test_questions, 1):
    dispute_type = classify_dispute_type(question)
    print(f"질문 {i}: {question}")
    print(f"분쟁 유형: {dispute_type}")
    print("-" * 40)


질문 1: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임
----------------------------------------
질문 2: 인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?
분쟁 유형: 이러닝
----------------------------------------
질문 3: 무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?
분쟁 유형: 이러닝
----------------------------------------
질문 4: 미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?
분쟁 유형: 게임
----------------------------------------
질문 5: 온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?
분쟁 유형: 웹콘텐츠
----------------------------------------
질문 6: 길드에서 비매너 행위로 강퇴 당했습니다. 다시 복귀 할 수 있을까요?
분쟁 유형: 게임
----------------------------------------
